In [43]:
# Import necessary libraries
import pandas as pd # data manipulation
import numpy as np # numerical python - linear algebra

from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [44]:
# load the dataset
df = pd.read_csv('afa2e701598d20110228.csv', sep=';')
df

,id,date,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,CL
0,1,17.02.2000,0.330,2.77,12.0,12.30,9.50,0.057,154.00,0.454,289.50
1,1,11.05.2000,0.044,3.00,51.6,14.61,17.75,0.034,352.00,0.090,1792.00
2,1,11.09.2000,0.032,2.10,24.5,9.87,13.80,0.173,416.00,0.200,2509.00
3,1,13.12.2000,0.170,2.23,35.6,12.40,17.13,0.099,275.20,0.377,1264.00
4,1,02.03.2001,0.000,3.03,48.8,14.69,10.00,0.065,281.60,0.134,1462.00
...,...,...,...,...,...,...,...,...,...,...,...
2856,22,06.10.2020,0.046,2.69,3.6,8.28,3.80,0.038,160.00,0.726,77.85
2857,22,27.10.2020,0.000,1.52,0.5,11.26,0.56,0.031,147.20,0.634,71.95
2858,22,03.12.2020,0.034,0.29,0.8,11.09,2.58,0.042,209.92,0.484,61.17
2859,22,12.01.2021,0.000,2.10,0.0,14.31,3.94,0.034,121.60,0.424,63.49


In [10]:
# Check and handle missing values
missing_values = df.isnull().sum()
print("Missing values before cleaning:\n", missing_values)

Missing values before cleaning:
 id            0
date          0
NH4           3
BSK5          1
Suspended    16
O2            3
NO3           1
NO2           3
SO4          49
PO4          28
CL           49
dtype: int64


In [13]:
# Confirm all missing values are handled
print("\nMissing values after cleaning:\n", df.isnull().sum())


Missing values after cleaning:
 id            0
date          0
NH4           3
BSK5          1
Suspended    16
O2            3
NO3           1
NO2           3
SO4          49
PO4          28
CL           49
dtype: int64


In [24]:
# Display column names to confirm structure
print("Columns in dataset:\n", df.columns)

Columns in dataset:
 Index(['id;date;NH4;BSK5;Suspended;O2;NO3;NO2;SO4;PO4;CL'], dtype='object')


In [28]:
#Show cleaned dataset shape and first few rows
print("\nDataset shape:", df.shape)
print(df.head())



Dataset shape: (2861, 1)
    id;date;nh4;bsk5;suspended;o2;no3;no2;so4;po4;cl
0  1;17.02.2000;0.33;2.77;12;12.3;9.5;0.057;154;0...
1  1;11.05.2000;0.044;3;51.6;14.61;17.75;0.034;35...
2  1;11.09.2000;0.032;2.1;24.5;9.87;13.8;0.173;41...
3  1;13.12.2000;0.17;2.23;35.6;12.4;17.13;0.099;2...
4  1;02.03.2001;0;3.03;48.8;14.69;10;0.065;281.6;...


In [31]:
# Check columns
print("Columns:", df.columns.tolist())


Columns: ['id;date;nh4;bsk5;suspended;o2;no3;no2;so4;po4;cl']


In [33]:
df.info() # dataset info

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2861 entries, 0 to 2860
Data columns (total 1 columns):
 #   Column                                            Non-Null Count  Dtype 
---  ------                                            --------------  ----- 
 0   id;date;nh4;bsk5;suspended;o2;no3;no2;so4;po4;cl  2861 non-null   object
dtypes: object(1)
memory usage: 22.5+ KB


In [36]:
# Statistics of the data
df.describe().T

,count,unique,top,freq
id;date;nh4;bsk5;suspended;o2;no3;no2;so4;po4;cl,2861,2861,1;17.02.2000;0.33;2.77;12;12.3;9.5;0.057;154;0...,1


In [37]:
# Missing values
df.isnull().sum()

id;date;nh4;bsk5;suspended;o2;no3;no2;so4;po4;cl    0
dtype: int64

In [41]:
# rows and cols
df.shape

(2861, 1)

In [45]:
# date is in object - date format
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y')
df


,id,date,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,CL
0,1,2000-02-17,0.330,2.77,12.0,12.30,9.50,0.057,154.00,0.454,289.50
1,1,2000-05-11,0.044,3.00,51.6,14.61,17.75,0.034,352.00,0.090,1792.00
2,1,2000-09-11,0.032,2.10,24.5,9.87,13.80,0.173,416.00,0.200,2509.00
3,1,2000-12-13,0.170,2.23,35.6,12.40,17.13,0.099,275.20,0.377,1264.00
4,1,2001-03-02,0.000,3.03,48.8,14.69,10.00,0.065,281.60,0.134,1462.00
...,...,...,...,...,...,...,...,...,...,...,...
2856,22,2020-10-06,0.046,2.69,3.6,8.28,3.80,0.038,160.00,0.726,77.85
2857,22,2020-10-27,0.000,1.52,0.5,11.26,0.56,0.031,147.20,0.634,71.95
2858,22,2020-12-03,0.034,0.29,0.8,11.09,2.58,0.042,209.92,0.484,61.17
2859,22,2021-01-12,0.000,2.10,0.0,14.31,3.94,0.034,121.60,0.424,63.49


In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2861 entries, 0 to 2860
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id         2861 non-null   int64         
 1   date       2861 non-null   datetime64[ns]
 2   NH4        2858 non-null   float64       
 3   BSK5       2860 non-null   float64       
 4   Suspended  2845 non-null   float64       
 5   O2         2858 non-null   float64       
 6   NO3        2860 non-null   float64       
 7   NO2        2858 non-null   float64       
 8   SO4        2812 non-null   float64       
 9   PO4        2833 non-null   float64       
 10  CL         2812 non-null   float64       
dtypes: datetime64[ns](1), float64(9), int64(1)
memory usage: 246.0 KB


In [59]:
df.describe()

,id,date,NH4,BSK5,Suspended,O2,NO3,NO2,SO4,PO4,CL,year,month
count,2861.000000,2861,2858.000000,2860.000000,2845.000000,2861.000000,2860.000000,2858.000000,2812.000000,2833.000000,2812.000000,2861.000000,2861.000000
mean,12.397064,2010-05-02 22:53:33.701502976,0.758734,4.316182,12.931905,9.508902,4.316846,0.246128,59.362313,0.418626,93.731991,2009.843411,6.540370
min,1.000000,2000-01-10 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,2000.000000,1.000000
25%,8.000000,2005-03-10 00:00:00,0.080000,2.160000,6.000000,7.100000,1.390000,0.030000,27.052500,0.130000,26.800000,2005.000000,4.000000
50%,14.000000,2010-06-04 00:00:00,0.220000,3.800000,10.000000,9.000000,2.800000,0.059000,37.800000,0.270000,33.900000,2010.000000,7.000000
75%,16.000000,2015-08-11 00:00:00,0.500000,5.800000,15.000000,11.520000,5.582500,0.125750,64.640000,0.470000,45.607500,2015.000000,9.000000
max,22.000000,2021-04-07 00:00:00,39.427000,50.900000,595.000000,90.000000,133.400000,109.000000,3573.400000,13.879000,5615.280000,2021.000000,12.000000
std,6.084226,NaN,2.486247,2.973997,16.543097,4.425937,6.881188,2.182777,96.582641,0.771326,394.512184,6.022041,3.344685


In [68]:
df = df.sort_values(by=['id', 'date'])
df.head()


,id,date,nh4,bsk5,suspended,o2,no3,no2,so4,po4,cl,year,month
3,2,2000-07-04,0.165,1.36,3.2,9.12,2.10,0.037,86.00,0.470,44.0,2000,7
4,2,2004-08-10,0.028,2.50,9.5,8.00,3.08,0.038,94.00,0.400,51.3,2004,8
6,2,2005-10-25,0.049,1.84,3.6,9.14,4.01,0.038,56.32,0.490,37.9,2005,10
7,2,2008-04-10,0.180,1.85,5.6,9.79,5.93,0.057,81.28,0.061,41.6,2008,4
9,2,2010-07-07,0.020,0.83,11.6,8.12,4.38,0.034,60.80,0.550,36.9,2010,7


In [69]:
# Missing values per column
print("Missing Values:\n", df.isnull().sum())

# Data types of each column
print("\nData Types:\n", df.dtypes)


Missing Values:
 id           0
date         0
nh4          1
bsk5         0
suspended    2
o2           0
no3          0
no2          0
so4          0
po4          2
cl           0
year         0
month        0
dtype: int64

Data Types:
 id                    int64
date         datetime64[ns]
nh4                 float64
bsk5                float64
suspended           float64
o2                  float64
no3                 float64
no2                 float64
so4                 float64
po4                 float64
cl                  float64
year                  int32
month                 int32
dtype: object


In [70]:
# Check frequency of values in a categorical column
print(df['month'].value_counts())


month
10    78
3     76
9     74
6     64
2     62
5     61
11    60
4     57
8     55
12    55
1     51
7     42
Name: count, dtype: int64


In [71]:
# Get correlation matrix
corr_matrix = df.corr(numeric_only=True)
print("\nCorrelation Matrix:\n", corr_matrix)



Correlation Matrix:
                  id       nh4      bsk5  suspended        o2       no3  \
id         1.000000  0.259035  0.192198   0.237827  0.003456  0.185618   
nh4        0.259035  1.000000  0.304997   0.249769 -0.023962  0.244376   
bsk5       0.192198  0.304997  1.000000   0.079671 -0.098219  0.230927   
suspended  0.237827  0.249769  0.079671   1.000000 -0.094107  0.064750   
o2         0.003456 -0.023962 -0.098219  -0.094107  1.000000  0.048428   
no3        0.185618  0.244376  0.230927   0.064750  0.048428  1.000000   
no2        0.171857  0.302972  0.221602   0.095760 -0.040734  0.295870   
so4       -0.075445  0.007881 -0.289496  -0.133538  0.164369 -0.030271   
po4        0.088997  0.508059  0.185900   0.083538 -0.054301  0.151455   
cl         0.030275  0.256047  0.027724  -0.040882  0.130948  0.178647   
year       0.087520  0.119121  0.212918  -0.082547  0.000813  0.186706   
month     -0.033265 -0.023876 -0.026435   0.071982 -0.051233 -0.204112   

               

In [72]:
# Skewness (asymmetry)
print("Skewness:\n", df.skew(numeric_only=True))

# Kurtosis (peakedness)
print("\nKurtosis:\n", df.kurt(numeric_only=True))


Skewness:
 id          -0.275236
nh4          6.582811
bsk5         1.930894
suspended    3.051132
o2          -0.119957
no3          2.022629
no2          7.274110
so4          1.636373
po4          6.341438
cl           1.522738
year        -0.017903
month       -0.023299
dtype: float64

Kurtosis:
 id           -0.743972
nh4          56.864764
bsk5          8.363478
suspended    16.150443
o2           -1.100273
no3           5.283146
no2          68.020586
so4           2.854794
po4          58.862813
cl            4.030460
year         -1.087293
month        -1.280193
dtype: float64


In [73]:
# Unique values in each column
for col in df.columns:
    print(f"{col}: {df[col].nunique()} unique values")

# Min/Max for selected features
print("\nNO3 range:", df['no3'].min(), "to", df['no3'].max())
print("O2 range:", df['o2'].min(), "to", df['o2'].max())


id: 21 unique values
date: 382 unique values
nh4: 289 unique values
bsk5: 252 unique values
suspended: 148 unique values
o2: 198 unique values
no3: 324 unique values
no2: 198 unique values
so4: 500 unique values
po4: 231 unique values
cl: 389 unique values
year: 22 unique values
month: 12 unique values

NO3 range: 2.02 to 23.7
O2 range: 6.02 to 9.97
